## openai 初始化

In [ ]:
%pip install openai
%pip install python-dotenv
%pip install pydantic

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
# print(f"OpenAI API Key: {openai_api_key}")

In [2]:
from openai import OpenAI
client = OpenAI()


# 廢話留言tag

### 第一層級的tag
instruction: "你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。"

In [9]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

def useless_comment_test1(text):
    instructions = """
    你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。
    只能用繁體中文回答

    <output>
    output as a json object:
    {
        "tag": bool,  # true=無意義、false=有意義
        "reason": str  # 無意義留言的原因
    }
    """

    class CommentAnalysis(BaseModel):
        tag: bool  # 是否為無意義留言
        reason: str  # 無意義留言的原因

    try:
        response = client.responses.parse(
            model="gpt-4.1-nano-2025-04-14",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": text}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "tag": result.tag,
            "reason": result.reason
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

In [10]:
## main function
def test1(comment_list):
    test1_result = []
    for comment in comment_list:
        print(f"Comment: {comment}")
        result = useless_comment_test1(comment)
        print(f"Result: {result}\n")

        temp_dict = {
            "comment_text" : comment,
            "tag": result['tag'],
            "reason": result['reason']
        }
        test1_result.append(temp_dict)

    return test1_result

In [11]:
comment_list = [
    '大陆官员都是韩国俞',
    '谁会想到 王牌变成了新闻乱报',
    '第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治',
    '求 女生 IG FB',
    '等等 那張車馬費我想要',
    '攝影機大哥是不是只故意拍妹子XD',
    '知聰明近乎勇',
    '模仿原汁那一段真是原汁原味',
    '結尾call back太好笑了',
    '打老二 跟大富翁好好笑',
    '檢查無效 司法有理',
    '結尾好屌',
    '結果天上掉下一席立委給元之',
    '真好笑 幫App廣告 不忘酸一下政治人物 那請問 你們酸人家 有分他廣告費嗎 那天 有人酸 後籠 賣產品 你應該也不會跟他拿 遮羞費吧 哈哈'
]

result = test1(comment_list)

Comment: 大陆官员都是韩国俞
{'tag': True, 'reason': '留言內容缺乏具體意義或具體的討論點，僅為一句模糊的描述，沒有提供實質資訊或有意義的內容。'}
Result: {'tag': True, 'reason': '留言內容缺乏具體意義或具體的討論點，僅為一句模糊的描述，沒有提供實質資訊或有意義的內容。'}

Comment: 谁会想到 王牌变成了新闻乱报
{'tag': False, 'reason': '留言內容有意涵，提及王牌變成新聞，具有明確的意思和背景信息。'}
Result: {'tag': False, 'reason': '留言內容有意涵，提及王牌變成新聞，具有明確的意思和背景信息。'}

Comment: 第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治
{'tag': False, 'reason': '留言內容涉及對電視節目的回顧及期待，有一定的意義與主題討論，並非無意義留言。'}
Result: {'tag': False, 'reason': '留言內容涉及對電視節目的回顧及期待，有一定的意義與主題討論，並非無意義留言。'}

Comment: 求 女生 IG FB
{'tag': True, 'reason': '內容只提及社交媒體帳號，缺乏具體或有意義的資訊'}
Result: {'tag': True, 'reason': '內容只提及社交媒體帳號，缺乏具體或有意義的資訊'}

Comment: 等等 那張車馬費我想要


KeyboardInterrupt: 

In [ ]:
# save result as csv
import pandas as pd

test1_df = pd.DataFrame(result)
# filename = f"gpt_tag/useless_tag_v{i}.csv"
filename = "gpt_tag/useless_test1.csv"
test1_df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Results saved to {filename}")

Results saved to gpt_tag/useless_test1.csv


### 第二層級的tag
instruction加上pseudo labeling


In [21]:
def useless_comment_test2(text):
    instructions = """
    你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。
    只能用繁體中文回答

    <definition>
    無意義留言是指那些沒有實質內容、無法提供有價值的資訊或討論的留言。
    </definition>

    <examples>
    - "求 女生 IG FB",  tag=true
    - "可憐的奮蛆們", tag=true
    - "真的 不好笑", tag=true
    - "賴的老家是國民黨遷台之前就有的 你們這節目一直批評賴的老家有沒有良心", tag=flase
    - "網路投票就不知道是不是個人意願啦 也沒辦法做到不公開吧", tag=false 
    - "所以說前幾年說要發放的數位身分證在哪裡呢", tag=false
    </examples>

    <output>
    output as a json object:
    {
        "tag": bool,  # true=無意義、false=有意義
        "reason": str  # 無意義留言的原因
    }
    """

    class CommentAnalysis(BaseModel):
        tag: bool  # 是否為無意義留言
        reason: str  # 無意義留言的原因

    try:
        response = client.responses.parse(
            model="gpt-4.1-nano-2025-04-14",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": text}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "tag": result.tag,
            "reason": result.reason
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

In [23]:
## main function
def test2(comment_list):
    test2_result = []
    for comment in comment_list:
        print(f"Comment: {comment}")
        result = useless_comment_test2(comment)
        print(f"Result: {result}\n")

        temp_dict = {
            "comment_text" : comment,
            "tag": result['tag'],
            "reason": result['reason']
        }
        test2_result.append(temp_dict)

    return test2_result

In [24]:
# test comments
comment_list = [
    '大陆官员都是韩国俞',
    '谁会想到 王牌变成了新闻乱报',
    '第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治',
    '求 女生 IG FB',
    '等等 那張車馬費我想要',
    '攝影機大哥是不是只故意拍妹子XD',
    '知聰明近乎勇',
    '模仿原汁那一段真是原汁原味',
    '結尾call back太好笑了',
    '打老二 跟大富翁好好笑',
    '檢查無效 司法有理',
    '結尾好屌',
    '結果天上掉下一席立委給元之',
    '真好笑 幫App廣告 不忘酸一下政治人物 那請問 你們酸人家 有分他廣告費嗎 那天 有人酸 後籠 賣產品 你應該也不會跟他拿 遮羞費吧 哈哈'
]

result2 = test2(comment_list)

Comment: 大陆官员都是韩国俞
{'tag': True, 'reason': '留言內容缺乏具體資訊或討論點，屬於無意義留言。'}
Result: {'tag': True, 'reason': '留言內容缺乏具體資訊或討論點，屬於無意義留言。'}

Comment: 谁会想到 王牌变成了新闻乱报
{'tag': False, 'reason': '留言具有討論性，涉及對某事件或情況的看法，具有一定的意義'}
Result: {'tag': False, 'reason': '留言具有討論性，涉及對某事件或情況的看法，具有一定的意義'}

Comment: 第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治
{'tag': False, 'reason': '留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。'}
Result: {'tag': False, 'reason': '留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。'}

Comment: 求 女生 IG FB
{'tag': True, 'reason': '留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。'}
Result: {'tag': True, 'reason': '留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。'}

Comment: 等等 那張車馬費我想要
{'tag': False, 'reason': '留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。'}
Result: {'tag': False, 'reason': '留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。'}

Comment: 攝影機大哥是不是只故意拍妹子XD
{'tag': False, 'reason': '留言具有討論內容，關於攝影和拍攝對象的話題，具有一定的意義。'}
Result: {'tag': False, 'reason': '留言具有討論內容，關於攝影和拍攝對象的話題，具有一定的意義。'}

Comment: 知聰明近乎勇
{'tag': False, 'reason': '該留言具有一定的哲理意義，表達了智慧與勇氣的關聯性，屬於有意義的留言。'}
Result: {'tag': 

In [27]:
# add tag result to useless_test1.csv, add column "tag2" and "reason2"
test2_df = pd.DataFrame(result2)
test2_df.rename(columns={"tag": "tag2", "reason": "reason2"}, inplace=True)
# test2_df.head(5)

# combine test1_df and test2_df
combined_df = pd.merge(test1_df, test2_df, on="comment_text", how="outer")
combined_df.head(5)

,comment_text,tag,reason,tag2,reason2
0,大陆官员都是韩国俞,True,留言內容不具備明確的意義，似乎為不相關或無意義的陳述。,True,留言內容缺乏具體資訊或討論點，屬於無意義留言。
1,谁会想到 王牌变成了新闻乱报,False,留言內容具有一定程度的意義，表達對某事件或情況的看法或感受。,False,留言具有討論性，涉及對某事件或情況的看法，具有一定的意義
2,第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治,False,留言內容表達了對節目的喜好與期待，具有一定的意義和評論性。,False,留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。
3,求 女生 IG FB,True,留言內容簡單，沒有提供具體資訊或有建設性的內容，屬於無意義留言。,True,留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。
4,等等 那張車馬費我想要,False,留言內容涉及對車馬費的需求，具有特定意義和實質內容。,False,留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。


### 第三層級的tag
instruction加上指標，reasonu要說明符合哪些指標

In [32]:
def useless_comment_test3(text):
    instructions = """
    你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。
    只能用繁體中文回答

    <definition>
    無意義留言的指標：
    1. 留言少於5個字
    2. 留言內容沒有意義 e.g. 哈哈哈笑死、真的 不好笑
    3. 留言內容重複 e.g. 主持人出來道歉 歧視弱勢團體 罷看 主持人出來道歉 歧視弱勢團體 罷看...
    4. 情緒性強或引發反感 e.g. 可憐的奮蛆們、賀龍神精病三八女、廢物綠蛆 廢物藍叫
    </definition>

    <examples>
    - "求 女生 IG FB",  tag=true
    - "可憐的奮蛆們", tag=true
    - "主持人出來道歉 歧視弱勢團體 罷看 主持人出來道歉 歧視弱勢團體 罷看...", tag=true
    - "賴的老家是國民黨遷台之前就有的 你們這節目一直批評賴的老家有沒有良心", tag=flase
    - "網路投票就不知道是不是個人意願啦 也沒辦法做到不公開吧", tag=false 
    - "所以說前幾年說要發放的數位身分證在哪裡呢", tag=false
    </examples>

    <task>根據definition和examples，分析留言是否為無意義留言</task>
    <dos>
    1. tag: 判斷留言是否符合無意義留言的定義
    2. reason: 說明留言符合哪些definition的指標
    </dos>

    <output>
    output as a json object:
    {
        "tag": bool,  # true=無意義、false=有意義
        "reason": str  # 無意義留言的原因
    }
    """

    class CommentAnalysis(BaseModel):
        tag: bool  # 是否為無意義留言
        reason: str  # 無意義留言的原因

    try:
        response = client.responses.parse(
            model="gpt-4.1-nano-2025-04-14",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": text}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "tag": result.tag,
            "reason": result.reason
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

In [33]:
## main function
def test3(comment_list):
    test3_result = []
    for comment in comment_list:
        print(f"Comment: {comment}")
        result = useless_comment_test3(comment)
        print(f"Result: {result}\n")

        temp_dict = {
            "comment_text" : comment,
            "tag": result['tag'],
            "reason": result['reason']
        }
        test3_result.append(temp_dict)

    return test3_result

In [35]:
# test comments
comment_list = [
    '大陆官员都是韩国俞',
    '谁会想到 王牌变成了新闻乱报',
    '第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治',
    '求 女生 IG FB',
    '等等 那張車馬費我想要',
    '攝影機大哥是不是只故意拍妹子XD',
    '知聰明近乎勇',
    '模仿原汁那一段真是原汁原味',
    '結尾call back太好笑了',
    '打老二 跟大富翁好好笑',
    '檢查無效 司法有理',
    '結尾好屌',
    '結果天上掉下一席立委給元之',
    '真好笑 幫App廣告 不忘酸一下政治人物 那請問 你們酸人家 有分他廣告費嗎 那天 有人酸 後籠 賣產品 你應該也不會跟他拿 遮羞費吧 哈哈'
]

result3 = test3(comment_list)

Comment: 大陆官员都是韩国俞
{'tag': False, 'reason': '留言內容有意義，提到具體人物且表達觀點，不符合無意義留言的指標'}
Result: {'tag': False, 'reason': '留言內容有意義，提到具體人物且表達觀點，不符合無意義留言的指標'}

Comment: 谁会想到 王牌变成了新闻乱报
{'tag': False, 'reason': '留言內容有意義，表達對新聞報導的不滿或感慨，符合有意義留言的特徵'}
Result: {'tag': False, 'reason': '留言內容有意義，表達對新聞報導的不滿或感慨，符合有意義留言的特徵'}

Comment: 第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治
{'tag': False, 'reason': '留言內容較完整，表達對節目的期待與評論，符合有意義留言的範圍'}
Result: {'tag': False, 'reason': '留言內容較完整，表達對節目的期待與評論，符合有意義留言的範圍'}

Comment: 求 女生 IG FB
{'tag': True, 'reason': '留言內容少於5個字，沒有明確的意義，屬於無意義留言'}
Result: {'tag': True, 'reason': '留言內容少於5個字，沒有明確的意義，屬於無意義留言'}

Comment: 等等 那張車馬費我想要
{'tag': True, 'reason': '留言內容較短，但沒有明顯的重複或情緒性，符合無意義留言的範例，且內容較為模糊不具意義。'}
Result: {'tag': True, 'reason': '留言內容較短，但沒有明顯的重複或情緒性，符合無意義留言的範例，且內容較為模糊不具意義。'}

Comment: 攝影機大哥是不是只故意拍妹子XD
{'tag': False, 'reason': '留言長度超過5個字，內容涉及對攝影師行為的質疑，具有一定意義，不符合無意義留言的定義。'}
Result: {'tag': False, 'reason': '留言長度超過5個字，內容涉及對攝影師行為的質疑，具有一定意義，不符合無意義留言的定義。'}

Comment: 知聰明近乎勇
{'tag': True, 'reason': '留

In [36]:
# add tag result to useless_test1.csv, add column "tag2" and "reason2"
test3_df = pd.DataFrame(result3)
test3_df.rename(columns={"tag": "tag3", "reason": "reason3"}, inplace=True)
# test2_df.head(5)

# combine test1_df and test2_df
combined_df = pd.merge(combined_df, test3_df, on="comment_text", how="outer")
combined_df.head(5)

,comment_text,tag,reason,tag2,reason2,tag3,reason3
0,大陆官员都是韩国俞,True,留言內容不具備明確的意義，似乎為不相關或無意義的陳述。,True,留言內容缺乏具體資訊或討論點，屬於無意義留言。,False,留言內容有意義，提到具體人物且表達觀點，不符合無意義留言的指標
1,谁会想到 王牌变成了新闻乱报,False,留言內容具有一定程度的意義，表達對某事件或情況的看法或感受。,False,留言具有討論性，涉及對某事件或情況的看法，具有一定的意義,False,留言內容有意義，表達對新聞報導的不滿或感慨，符合有意義留言的特徵
2,第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治,False,留言內容表達了對節目的喜好與期待，具有一定的意義和評論性。,False,留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。,False,留言內容較完整，表達對節目的期待與評論，符合有意義留言的範圍
3,求 女生 IG FB,True,留言內容簡單，沒有提供具體資訊或有建設性的內容，屬於無意義留言。,True,留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。,True,留言內容少於5個字，沒有明確的意義，屬於無意義留言
4,等等 那張車馬費我想要,False,留言內容涉及對車馬費的需求，具有特定意義和實質內容。,False,留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。,True,留言內容較短，但沒有明顯的重複或情緒性，符合無意義留言的範例，且內容較為模糊不具意義。


In [37]:
combined_df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Results saved to {filename}")

Results saved to gpt_tag/useless_test1.csv


In [ ]:
import pandas as pd
import asyncio
from gpt_instructions import *
from openai import AsyncOpenAI
from pydantic import BaseModel

# 非同步 client
client = AsyncOpenAI()

file_path = "gpt_tag/video_0_useless_tag.csv"

# 定義 Pydantic 類別
class CommentAnalysis(BaseModel):
    tag: bool
    reason: str

# 非同步處理單一 prompt
async def get_useless_response(text, instructions_num):
    try:
        response = await client.chat.completions.create(
            model="gpt-4.1-nano-2025-04-14",
            messages=[
                {"role": "system", "content": instructions_num},
                {"role": "user", "content": text}
            ],
            response_format= {
                "type": "json_object",
            }
        )
        result = CommentAnalysis.model_validate_json(response.choices[0].message.content)
        return {"tag": result.tag, "reason": result.reason}

    except Exception as e:
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

# 主處理函式：並行執行三組指令
async def process_comments():
    comment_df = pd.read_csv(file_path, encoding='utf-8-sig')[:10]
    async_test_df = comment_df.copy()

    for idx, row in comment_df.iterrows():
        comment_text = row['comment_text']
        print(f"==> 處理第{idx+1}則留言：{comment_text}")

        # 並行處理三組 instructions
        result1, result2, result3 = await asyncio.gather(
            get_useless_response(comment_text, instruction1),
            get_useless_response(comment_text, instruction2),
            get_useless_response(comment_text, instruction3),
        )

        # 更新DataFrame
        async_test_df.at[idx, 'tag1'] = result1['tag']
        async_test_df.at[idx, 'reason1'] = result1['reason']
        async_test_df.at[idx, 'tag2'] = result2['tag']
        async_test_df.at[idx, 'reason2'] = result2['reason']
        async_test_df.at[idx, 'tag3'] = result3['tag']
        async_test_df.at[idx, 'reason3'] = result3['reason']

    # 儲存結果
    async_test_df.to_csv("gpt_tag/video_0_async_tag.csv", index=False, encoding='utf-8-sig')
    print(async_test_df.head(5))

In [21]:
# 同步處理
await process_comments()

處理留言：太有活了贵司
處理留言：我的天这段影片的评论区是发生了什么
處理留言：原来这就是之后节目里提到的被炎上的事件之一吗
處理留言：突然發現低卡的夥伴們ᶘ ᵒᴥᵒᶅ
處理留言：怎麼選舉可以有這麼多好笑的內容
處理留言：桌上是什麼怎麼變成馬賽克了
處理留言：王志安事件
處理留言：原本放什麼
處理留言：為什麼桌上的廠商有打碼 節目結束的廠商沒有打碼
處理留言：藍綠白都臭 只有綠不爽 可見其風度
             comment_text   tag1                          reason1   tag2  \
0                  太有活了贵司   True     內容模糊，不具具體意義或資訊，無法提供有效的反饋或互動。   True   
1       我的天这段影片的评论区是发生了什么  False  留言內容具有疑問或關心影片內容，有意圖交流，不屬於無意義留言。   True   
2  原来这就是之后节目里提到的被炎上的事件之一吗  False         內容具有搜尋或討論的價值，沒有顯示為無意義留言。  False   
3        突然發現低卡的夥伴們ᶘ ᵒᴥᵒᶅ  False      留言內容有表情符號和情感，傳達特定意圖，具有一定的意義   True   
4         怎麼選舉可以有這麼多好笑的內容  False   留言表達對選舉內容的看法，有一定意義，並非純粹的無意義內容。   True   

                               reason2   tag3  \
0           留言內容沒有具體資訊或討論點，屬於模糊或無意義的評論   True   
1  留言內容沒有具體內容或有意義的討論，僅表達困惑或感受，屬於無意義留言。  False   
2   留言內容具有疑問性，涉及對未來節目內容的理解與關注，具備一定討論價值  False   
3            內容缺乏具體資訊或討論價值，只是簡單的表情和口號。   True   
4         留言內容較為空泛，未提供具體資訊或討論，屬於無意義留言。  False   

                                          reason3

In [6]:
# 計算標註一致性

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 範例資料
true_labels = human_tag_df['human_tag']
pred_labels = human_tag_df['spam_tag']

# 計算四個指標（macro平均適用於不平衡資料）
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')
accuracy = accuracy_score(true_labels, pred_labels)

print('Precision（精確率）:', precision)
print('Recall（召回率）:', recall)
print('F1 Score:', f1)
print('Accuracy（準確率）:', accuracy)

# 儲存結果
results = {
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "accuracy": accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv("batch_outputs/llm_tagging_metrics.csv", index=False, encoding='utf-8-sig')

Precision（精確率）: 0.845
Recall（召回率）: 0.8672228585053072
F1 Score: 0.8426189825875673
Accuracy（準確率）: 0.845


# 影片相關性

In [37]:
# test 10則

with open('transcripts/v0.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

print(transcript)

歡迎收看《賀瓏夜夜秀》
再次感謝冷靜寶貝樂團
各位觀眾
海水退了
沒穿褲子的人
來點尖叫聲
什麼東西啊
《賀瓏夜夜秀》
到底是哪一個黨派哪一個立場呢
你們覺得呢
你們覺得
我們挺誰呢
當然是
TEAM TAIWAN 啊
看不出來嗎
一路以來
我們都是支持賴蕭配的嘛
對不對
接下來四年
請好好照顧我們
我們就是綠蛆秀啊
恭喜賴清德拿下 558 萬票
成為 2024 年的台灣新總統
讓我們來聽聽
賴清德的勝選感言
大家有仔細看清楚嗎
賴清德一當選
就換了新髮型耶
可是
讓台灣成為世界關鍵字
真的好嗎
去年全球 Google 關鍵字
第一名是以巴戰爭
誰要當啊
但台灣兩週前的關鍵字
也差不多
叫做 Missile
衛星的名字
那個英文字
明明就是飛彈
欸 國防部
真的菜英文耶
但一樣都是勝選
賴清德意氣風發
蕭美琴
就笑不出來
這樣的副手不及格喔
總統講什麼
都要笑啊
蕭美琴臉這麼臭
是因為想回去發 Theards 嗎
民進黨這禮拜
攻佔了 Theards
尤其是剛落選的
鄭運鵬
是怕知名度輸給兒子嗎
民進黨的勝選之夜
蕭美琴臉很臭
但國民黨的敗選晚會
全場臉都很臭
發言人楊智伃哭得好慘
好讓人心疼喔
沒有辦法
他才 27 歲
太年輕了
其他老人沒有哭
是因為他們都輸習慣了
國民黨也太擅長
讓發言人哭了吧
很可惜馬英九沒有出席敗選晚會
因為國民黨不歡迎他
馬英九還是當見證人就好
沒有想到馬英九
這麼快就成為了
失智老藍男
年紀也到了啦
他自己
也沒想到有今天吧
不過口誤大前輩趙少康
還是很照顧馬英九
一本書不夠組讀書會啦
我們可以再送馬英九一本
習近平自傳
下架國民黨
相信習近平
這是我和馬英九推出的新年春聯
他們組完讀書會
可以再找柯文哲
組一個
選輸會
哇 妳不要這樣講
民眾黨沒有輸
其實這場選戰真的沒有人輸
賴清德
當選總統
侯友宜
當選新北市長
趙少康
當選戰情室主持人
吳欣盈
當選
賀瓏杯杯代言人
說到做到 讚
而柯文哲呢
當選
新髮型代言人
有人加入髮型界的新潮流囉
民眾黨的敗選晚會上
柯文哲說了一句話
讓幕僚都傻眼
星期天還要上班
義無反顧
違反一次勞基法
太離譜了
黃國昌是不是後悔加入民眾黨啊
各位觀眾
《賀瓏夜夜秀》的選舉成績
訪談來賓
敗選率
高達
67%
我們被講成
敗選夜夜秀
欸 太難聽了吧
我們熊字輩的都有選上啊
黑熊 沈伯洋
小熊 徐巧芯
還有

In [73]:
import pandas as pd
comments_df = pd.read_csv("batch_outputs/usefull_comments.csv", encoding='utf-8-sig')
test_df = comments_df[:100]


In [72]:
video_comment_counts = comments_df.groupby('video_title').size().reset_index(name='comment_count')
print(video_comment_counts)

                        video_title  comment_count
0   【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會            902
1       【#賀瓏夜夜秀】1/27 新聞亂報 EP10｜新聞自報           7320
2       【#賀瓏夜夜秀】1/6 新聞亂報 EP8｜大選前不演了            752
3     【#賀瓏夜夜秀】10/28 新聞亂報 EP1｜藍白兩情相悅            630
4    【#賀瓏夜夜秀】11/18 新聞亂報 EP3｜沒想到有今天吧            570
5   【#賀瓏夜夜秀】11/25 新聞亂報 EP4｜你也可以不同意啊            854
6     【#賀瓏夜夜秀】11/4 新聞亂報 EP2｜藍白誰攻誰受？            563
7    【#賀瓏夜夜秀】12/16 新聞亂報 EP6｜Blue不錄了           1126
8      【#賀瓏夜夜秀】12/30 新聞亂報 EP7｜新北割侯戰           1102
9        【#賀瓏夜夜秀】12/9 新聞亂報 EP5｜財產亂報            702
10     【#賀瓏夜夜秀】吳欣岱、陳奕齊 最能靠美色競選的本土政黨           3347
11       【#賀瓏夜夜秀】吳欣盈 參選副總統其實有點Shock           5801
12         【#賀瓏夜夜秀】徐巧芯 認定國民黨的男人都很軟爛           1189
13                    【#賀瓏夜夜秀】欸！吹哨者           2827
14                    【#賀瓏夜夜秀】欸！外役監           1194
15                     【#賀瓏夜夜秀】欸！大麻            928
16                    【#賀瓏夜夜秀】欸！投開票           2991
17                    【#賀瓏夜夜秀】欸！特偵組           3142
18                   【#賀瓏夜夜秀】欸！

In [33]:
# 先判斷影片主題關鍵字，再判斷留言相關性？

def fetch_video_keywords(transcript):
    instruction = "只能用繁體中文回答。你是YouTube內容分析研究員，擅長主題分析。萃取10-15組<transcript>的具體關鍵字。關鍵字排除如，賀瓏夜夜秀、冷靜寶貝樂團，以及抽象形容，如：政治喜劇與社會關懷、香港社會"
    class TranscriptKeywords(BaseModel):
        transcript_keywords: list[str]  # 影片主題關鍵字
    try:
        response = client.responses.parse(
            model="gpt-4.1-mini-2025-04-14",
            input=[
                {"role": "system", "content": instruction},
                {"role": "user", "content": transcript}
            ],
            text_format=TranscriptKeywords
        )

        result = response.output_parsed
        print(f"Transcript Keywords: {result.transcript_keywords}")
        return result.transcript_keywords
    
    except Exception as e:
        print(f"Error: {e}")
        return []


In [38]:
transcript_keywords = fetch_video_keywords(transcript)

Transcript Keywords: ['賴清德勝選', '新總統', '賴蕭配', '綠蛆秀', '副手蕭美琴', '民進黨', '國民黨敗選', '馬英九', '趙少康', '柯文哲髮型', '民眾黨', '選舉成績', '立法院長競爭', '韓國瑜', '江啟臣', '施明德逝世', '諾魯斷交', '三小國家', '夜市科目三活動', 'IU演唱會']


In [2]:
def comment_video_related(comment, transcript_keywords, knowledge_list):
    instructions = f"""
    只能用繁體中文回答
    你是台灣YouTube留言的內容分析研究員，擅長主題分析，且熟悉台灣的網路政治討論。任務：分析留言是否與影片主題相關。
    
    {knowledge_list}是台灣網路上常見的政治用語 格式 “人物或單位（職稱、節目中角色）：別名”

    <task>判斷<comment>是否討論、回應、發表與{transcript_keywords}相關的內容，並給出理由<reason>。
    
    if related == true, 從transcript_keywords中找出與留言相關的關鍵字，並回傳match_keywords
    else: related為false，match_keywords為None

    <output>
    {{
        "related": bool,
        "reason": str,
        "match_keywords": list[str] | None
    }}
    """

    print(f">>> transcript_keywords 字數: {len(transcript_keywords)}")
    print(">>> knowledge字數:" "\"" + str(len(knowledge_list)) + "\"")
    print(">>> instruction字數:" "\"" + str(len(instructions)) + "\"")

    print(f">>> system prompt 總字數: {len(instructions) + len(transcript_keywords)+ len(knowledge_list)}")

    # class CommentAnalysis(BaseModel):
    #     related: bool  # 是否與影片相關
    #     reason: str  # 相關的理由
    #     match_keywords: list[str] | None  # 影片主題關鍵字
    
    # try:
    #     response = client.responses.parse(
    #         model="gpt-4.1-mini-2025-04-14",
    #         input=[
    #             {"role": "system", "content": instructions},
    #             {"role": "user", "content": comment}
    #         ],
    #         text_format=CommentAnalysis,
    #     )

    #     result = response.output_parsed

    #     result_dict = {
    #         "related": result.related,
    #         "reason": result.reason,
    #         "match_keywords": result.match_keywords
    #     }
    #     print(result_dict)
    #     return result_dict
    
    # except Exception as e:
    #     print(f"Error: {e}")
    #     return {"related": False, "reason": f"分析失敗: {str(e)}", "match_keywords": None}

In [75]:
# 使用 apply 方法，避免逐行迭代
def process_comment(row):
    comment_text = row['comment']
    print(f"==> 處理留言：{comment_text}")
    
    result = comment_video_related(comment_text, transcript_keywords)
    
    return pd.Series({
        'related': result['related'],
        'reason': result['reason'],
        'match_keywords': ', '.join(result['match_keywords']) if result['match_keywords'] else None
    })

# 應用函數到每一行
new_columns = test_df.apply(process_comment, axis=1)
test_df[['related', 'reason', 'match_keywords']] = new_columns

==> 處理留言：原来这就是之后节目里提到的被炎上的事件之一吗
{'related': False, 'reason': '留言提到被炎上的事件，但沒有明確提及與指定主題相關的內容，如賴清德勝選或其他政治人物和事件。無法確定與影片主題相關。', 'match_keywords': None}
==> 處理留言：怎麼選舉可以有這麼多好笑的內容
{'related': False, 'reason': '留言提及選舉有趣內容，但沒有具體涉及指定的主題關鍵字，如賴清德勝選或民進黨等，無法確定與影片主題相關。', 'match_keywords': None}
==> 處理留言：桌上是什麼怎麼變成馬賽克了
{'related': False, 'reason': '留言內容是詢問桌上物品為何成為馬賽克，並未討論或回應影片主題相關政治人物或事件。', 'match_keywords': None}
==> 處理留言：為什麼桌上的廠商有打碼 節目結束的廠商沒有打碼
{'related': False, 'reason': '留言內容是關於影片畫面中廠商名稱是否打碼的觀察，與指定的政治及娛樂主題無關。', 'match_keywords': None}
==> 處理留言：藍綠白都臭 只有綠不爽 可見其風度
{'related': True, 'reason': '留言評論涉及藍綠白三個政治陣營，與影片主題中提到的民進黨（綠營）、國民黨（藍營）及民眾黨（白營）相關，討論政治立場及風度，與影片政治主題相關。', 'match_keywords': ['民進黨', '國民黨', '民眾黨']}
==> 處理留言：還好吧 一堆藍白也很不爽說夜夜秀是側翼啊 真是見鬼了
{'related': True, 'reason': '留言提及“藍白”及“夜夜秀”暗示政治立場及節目名稱，與影片中涉及的政治主題“綠蛆秀”相關。', 'match_keywords': ['綠蛆秀']}
==> 處理留言：現在回來看 不愧是死亡之握
{'related': False, 'reason': '留言中的『死亡之握』並未直接提及或回應任何指定的主題關鍵詞，無法判定其與影片主題相關。', 'match_keywords': None}
==> 處理留言：蟹 這集有亮點捏
{'related': False, '

/var/folders/hs/7z23ptbn2v76419z9twk6nxc0000gn/T/ipykernel_25555/2715122244.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[['related', 'reason', 'match_keywords']] = new_columns


In [76]:
test_df.to_csv("test_v0_related.csv", index=False, encoding='utf-8-sig')